In [1]:
%load_ext autoreload
%autoreload 2
import pyccl as ccl
import CL_COUNT_Fisher_Matrix as fm
import CL_COUNT_class_cluster_abundance as cl_count
import matplotlib.pyplot as plt
import numpy as np
def binning(corner): return [[corner[i],corner[i+1]] for i in range(len(corner)-1)]

# True cosmology

In [2]:
Omega_c_true = 0.30711 - 0.048254
Omega_b_true = 0.048254
sigma8_true = .8288
Omegam_true = 0.30711
True_value = [Omega_c_true + Omega_b_true, sigma8_true]
cosmo_true = ccl.Cosmology(Omega_c = Omega_c_true, Omega_b = Omega_b_true, h = 0.6777, sigma8 = sigma8_true, n_s=0.96)

# Define Mass-redshift binning

In [3]:
n_logm_bin = 5
n_z_bin = 5
logm_corner = np.linspace(14, 15, n_logm_bin + 1)
logMass_bin = binning(logm_corner)
z_corner = np.linspace(0.1,2, n_z_bin + 1)
Z_bin = binning(z_corner)

# Define Fisher Matrix Object

In [4]:
FM = fm.Fisher_Matrix(cosmo_true)

## Initialize CLusterAbundance object

In [5]:
FM.set_ClusterAbundance_Object(f_sky = .25)

## Compute true quantities (N, covariances, halo baises, etc.)

In [6]:
FM.set_cosmo_definitions(cosmo_true)
FM.compute_grid(bais = True)
Abundance_true = FM.model_Binned_Nth(Z_bin, logMass_bin, cosmo_true)
Halo_bias_true = FM.CA.halo_bias_MZ(Redshift_bin = Z_bin, Proxy_bin = logMass_bin, N_th = Abundance_true)
cov_SSC_true = FM.CA.sample_covariance_MZ(Z_bin, logMass_bin, Abundance_true, Halo_bias_true)
cov_Shot_Noise_true = np.diag(Abundance_true.flatten())

# Binned Gaussian, Binned Gaussian (diagonal covariance), Binned Poissonian

In [7]:
cPoiss, cGauss_diag, cGauss = FM.Fisher_matrix_binned(Z_bin, logMass_bin, cosmo_true, Abundance_true.flatten(), cov_SSC_true, cov_Shot_Noise_true)

# Unbinned Poissonian

In [ ]:
cUnbinned = FM.Fisher_matrix_unbinned_Poissonian([0.1, 2], [14, 15], cosmo_true)

# Binned MVP

In [ ]:
cMVP = FM.Fisher_matrix_Binned_MVP(Z_bin, logMass_bin, cosmo_true, cov_SSC_true)

# Plot 2-$\sigma$ contours

In [ ]:
fig, ax = plt.subplots(figsize = (7,7))
mean = FM.True_value_0m_s8
print(mean)
from scipy import stats
from matplotlib.patches import Ellipse
def _make_ellipse(mean, cov, ax, level=0.95, color=None, label = None, ls = '-'):
    n_sigma = 2
    v, w = np.linalg.eigh(cov)
    u = w[0] / np.linalg.norm(w[0])
    angle = np.arctan(u[1] / u[0])
    angle = 180 * angle / np.pi 
    ell = Ellipse(mean, 2 * n_sigma * v[0] ** 0.5, 2 * n_sigma * v[1] ** 0.5,
                              180 + angle,
                              linewidth=2, facecolor = 'none',edgecolor=color, ls = ls)
    ax.plot([],[],ls,color = color, label = label)
    ax.add_artist(ell)
_make_ellipse(mean, cUnbinned, ax, color='b', label = r'Poisson Unbinned', ls = '--')
_make_ellipse(mean, cPoiss, ax, color='b', label = r'Poisson Binned', ls = '-')
_make_ellipse(mean, cGauss, ax, color='r', label = r'Gaussian Binned', ls = '-')
_make_ellipse(mean, cGauss_diag, ax, color='cyan', label = r'Gaussian Binned (diagonal)', ls = '-')
_make_ellipse(mean, cMVP, ax, color='g', label = r'MVP Binned', ls = '--')
std_Om = 0.0028
std_s8 = 0.0018
ax.set_xlim(mean[0] - std_Om, mean[0] + std_Om)
ax.set_ylim(mean[1] - std_s8, mean[1] + std_s8)
ax.vlines(mean[0], 0, 1, color = 'k', linestyle = '--')
ax.hlines(mean[1], 0, 1, color = 'k', linestyle = '--')
ax.legend(fontsize = 15, frameon = False)
ax.tick_params(axis='both', which = 'major', labelsize= 15)
parameters = [r'$\Omega_m$',r'$\sigma_8$']
ax.set_xlabel(parameters[0], fontsize = 30)
ax.set_ylabel(parameters[1], fontsize = 30)
#plt.savefig('/pbs/throng/lsst/users/cpayerne/ThesisAtCCin2p3/Images/forecast_5times5.pdf', bbox_inches='tight', dpi=300)